In [ ]:
import ibis
from decouple import config, AutoConfig

In [ ]:
con = ibis.connect("duckdb://penguins.ddb")

In [ ]:
csv_path = r'C:\Users\Zygim\Downloads\archive\weather_data.csv'
table_name = 'weather_data'

In [ ]:
con.raw_sql(f"""CREATE TABLE {table_name} AS SELECT * FROM read_csv_auto('{csv_path}');""")

In [ ]:
con.list_tables()

In [ ]:
con.raw_sql(f"""SELECT * FROM {table_name} LIMIT 10""").fetchdf()

In [ ]:
table = con.table(table_name)

In [ ]:
table.schema()

In [ ]:
con.raw_sql(f"""SELECT * FROM {table_name} LIMIT 10""").fetchdf()

In [ ]:
con.sql("""
  SELECT * FROM weather_data
""")

# Section 5 Remember

In [ ]:
config = AutoConfig(search_path=r'F:\DataSpell\data_engineering_essentials_using_sql_python_pyspark')

postgres = ibis.postgres.connect(
    host=config('HOST'),
    port=config('PORT'),
    user=config('USER'),
    password=config('PASSWORD'),
    database=config('DATABASE')
)

In [ ]:
postgres.list_tables()

In [ ]:
con.raw_sql(f"""SELECT * FROM weather_data""").fetchdf()

In [ ]:
con.raw_sql("""
    SELECT *
    FROM weather_data
    WHERE Location IN ('San Antonio', 'New York')
""").fetchdf()

In [ ]:
con.raw_sql("""
    SELECT count(DISTINCT Location)
    FROM weather_data
""").fetchdf()

In [10]:
con.raw_sql("""
    SELECT round(SUM(Precipitation_mm), 2)
    FROM weather_data
    WHERE Location = 'San Antonio'
""").fetchdf()

,"round(sum(Precipitation_mm), 2)"
0,500618.82


In [13]:
con.raw_sql("""
    SELECT * FROM weather_data
""").fetchdf()

,Location,Date_Time,Temperature_C,Humidity_pct,Precipitation_mm,Wind_Speed_kmh
0,San Diego,2024-01-14 21:12:46,10.683001,41.195754,4.020119,8.233540
1,San Diego,2024-05-17 15:22:10,8.734140,58.319107,9.111623,27.715161
2,San Diego,2024-05-11 09:30:59,11.632436,38.820175,4.607511,28.732951
3,Philadelphia,2024-02-26 17:32:39,-8.628976,54.074474,3.183720,26.367303
4,San Antonio,2024-04-29 13:23:51,39.808213,72.899908,9.598282,29.898622
...,...,...,...,...,...,...
999995,Dallas,2024-01-01 20:29:48,23.416877,37.705024,3.819833,16.538119
999996,San Antonio,2024-01-20 15:59:48,6.759080,40.731036,8.182785,29.005558
999997,New York,2024-04-14 08:30:09,15.664465,62.201884,3.987558,0.403909
999998,Chicago,2024-05-12 20:10:43,18.999994,63.703245,4.294325,6.326036


In [21]:
con.raw_sql("""
    SELECT Location, count(*) AS count_location
    FROM weather_data
    GROUP BY 1,
    order by 2 desc
""").fetchdf()

,Location,count_location
0,Phoenix,100209
1,Chicago,100164
2,Philadelphia,100122
3,Houston,100076
4,New York,99972
5,San Antonio,99962
6,Dallas,99936
7,Los Angeles,99922
8,San Jose,99863
9,San Diego,99774


In [29]:
con.raw_sql("""
    SELECT strftime(Date_Time, '%Y-%m'), count(*) AS count_date
    FROM weather_data
    GROUP BY 1
    ORDER BY 2 DESC
""").fetchdf()

,"strftime(Date_Time, '%Y-%m')",count_date
0,2024-01,223290
1,2024-03,223072
2,2024-04,216804
3,2024-02,208478
4,2024-05,128356


In [43]:
con.raw_sql("""
    SELECT
    Location,
    round(sum(Precipitation_mm), 2) AS total_precipitation
    FROM weather_data
    GROUP BY Location
    ORDER BY total_precipitation DESC
""").fetchdf()

,Location,total_precipitation
0,Phoenix,610020.81
1,Philadelphia,501467.10
2,Chicago,501136.93
3,San Antonio,500618.82
4,Houston,500155.54
5,Los Angeles,500058.77
6,San Diego,499527.82
7,Dallas,499430.93
8,New York,499165.85
9,San Jose,498056.57


In [4]:
%%sql
SELECT order_items.order_item_order_id,
    round(sum(order_items.order_item_subtotal)::numeric, 2) AS order_revenue
FROM order_items
GROUP BY 1
    HAVING round(sum(order_items.order_item_subtotal)::numeric, 2) >= 2000
    ORDER BY 2 DESC;

,order_item_order_id,order_revenue
0,68703,3449.91
1,68724,2859.89
2,68858,2839.91
3,68809,2779.86
4,68766,2699.90
5,68821,2629.92
6,68806,2629.92
7,68778,2629.90
8,68848,2399.96
9,68875,2399.95


In [21]:
%%sql
SELECT order_item_order_id,
    round(sum(order_item_subtotal)::numeric, 2) AS order_revenue
FROM order_items
GROUP BY 1
    HAVING ROUND(SUM(order_item_subtotal)::numeric, 2) >= 2500
ORDER BY order_revenue DESC;

,order_item_order_id,order_revenue
0,68703,3449.91
1,68724,2859.89
2,68858,2839.91
3,68809,2779.86
4,68766,2699.90
5,68806,2629.92
6,68821,2629.92
7,68778,2629.90


In [25]:
%%sql
SELECT
    o.order_date,
    oi.order_item_order_id,
    oi.order_item_subtotal
FROM orders AS o
    JOIN order_items AS oi
        ON o.order_id = oi.order_item_id;

,order_date,order_item_order_id,order_item_subtotal
0,2013-07-25 00:00:00.000000,1,299.98
1,2013-07-25 00:00:00.000000,2,199.99
2,2013-07-25 00:00:00.000000,2,250.00
3,2013-07-25 00:00:00.000000,2,129.99
4,2013-07-25 00:00:00.000000,4,49.98
...,...,...,...
68878,2014-07-09 00:00:00.000000,27521,150.00
68879,2014-07-13 00:00:00.000000,27521,119.98
68880,2014-07-19 00:00:00.000000,27521,199.99
68881,2014-07-22 00:00:00.000000,27521,150.00


# 45. Outer Joins using SQL Queries

In [10]:
%%sql
SELECT o.order_id,
       to_char(o.order_date, 'yyyy-DD-mm'),
       oi.order_item_id,
       oi.order_item_subtotal
FROM orders AS o
    LEFT OUTER JOIN order_items AS oi
        ON o.order_id = oi.order_item_id;

,order_id,to_char,order_item_id,order_item_subtotal
0,1,2013-25-07,1,299.98
1,2,2013-25-07,2,199.99
2,3,2013-25-07,3,250.00
3,4,2013-25-07,4,129.99
4,5,2013-25-07,5,49.98
...,...,...,...,...
68878,68879,2014-09-07,68879,150.00
68879,68880,2014-13-07,68880,119.98
68880,68881,2014-19-07,68881,199.99
68881,68882,2014-22-07,68882,150.00


In [12]:
%%sql
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';

,table_name
0,departments
1,categories
2,products
3,customers
4,orders
5,order_items
6,daily_product_revenue
7,order_details_v
8,order_count_by_status
9,orders_stg


In [ ]:
%%sql
DROP VIEW IF EXISTS order_details_v;

In [16]:
%%sql
CREATE OR REPLACE VIEW order_details_v
AS
SELECT o.*,
        oi.order_item_product_id,
        oi.order_item_subtotal,
        oi.order_item_id
from orders AS o
    JOIN order_items AS oi
        ON o.order_id = oi.order_item_order_id;

In [27]:
%%sql
select order_status, order_id from order_details_v
GROUP BY order_status, order_id
HAVING order_id < 3;

,order_status,order_id
0,CLOSED,1
1,PENDING_PAYMENT,2


In [29]:
%%sql
WITH order_details_cte AS
(SELECT *
FROM orders AS o
    JOIN order_items AS oi
        ON o.order_id = oi.order_item_id)
SELECT * FROM order_details_cte
WHERE order_id = 2;

,order_id,order_date,order_customer_id,order_status,order_item_id,order_item_order_id,order_item_product_id,order_item_quantity,order_item_subtotal,order_item_product_price
0,2,2013-07-25 00:00:00.000000,256,PENDING_PAYMENT,2,2,1073,1,199.99,199.99


In [31]:
%%sql
SELECT *
FROM products as p
    LEFT OUTER JOIN order_details_v AS odv
        ON p.product_id = odv.order_item_product_id
        AND to_char(odv.order_date::timestamp, 'yyyy-MM') = '2014-01'
    WHERE odv.order_item_product_id IS NULL;

,product_id,product_category_id,product_name,product_description,product_price,product_image,order_id,order_date,order_customer_id,order_status,order_item_product_id,order_item_subtotal,order_item_id
0,1074,48,Field & Stream Eagle Talon Kayak,NaN,549.99,http://images.acmesports.sports/Field+%26+Stre...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,887,40,Team Golf New York Yankees Putter Grip,NaN,24.99,http://images.acmesports.sports/Team+Golf+New+...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,264,12,Nike Dri-FIT Crew Sock 6 Pack,NaN,22.00,http://images.acmesports.sports/Nike+Dri-FIT+C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,802,36,Glove It Women's Olive Bling Golf Glove,NaN,19.99,http://images.acmesports.sports/Glove+It+Women...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1070,48,Coleman Scanoe Canoe,NaN,399.99,http://images.acmesports.sports/Coleman+Scanoe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1241,925,41,Glove It Mod Oval Golf Towel,NaN,15.99,http://images.acmesports.sports/Glove+It+Mod+O...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1242,612,41,Garmin vivofit Fitness Band with HRM,NaN,169.99,http://images.acmesports.sports/Garmin+vivofit...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1243,1119,50,Majestic Men's 2014 All-Star Game Yadier Moli,NaN,130.00,http://images.acmesports.sports/Majestic+Men%2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1244,616,41,Nike+ Sportwatch GPS Powered by TomTom with S,NaN,169.00,http://images.acmesports.sports/Nike%2B+Sportw...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
